In [ ]:
!conda install pyspark

In [ ]:
!conda install pyspark[connect]

In [16]:
!pip install pyarrow

In [4]:
import os
os.environ['HADOOP_HOME'] = 'C:/tools/hadoop-3.2.0'

In [2]:
import pandas as pd

In [3]:
#cause iteritems is deprecated in pandas 2 so replace with items instead of downgrade pandas
pd.DataFrame.iteritems = pd.DataFrame.items

In [4]:
pd.__version__

'2.1.4'


- SparkSession is an entrypoint for pyspark
- master() – If you are running it on the cluster you need to use your master name as an argument to master(). usually, 
it would be either yarn or mesos depends on your cluster setup.

Use local[x] when running in Standalone mode. x should be an integer value and should be greater than 0; this represents how many partitions it should create when using RDD, DataFrame, and Dataset. Ideally, x value should be the number of CPU cores you have.

- appName() – Used to set your application name.
- 
getOrCreate() – This returns a SparkSession object if already exists, and creates a new one if not exist.


In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

#read csv file as spark dataframe
df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

df.show()

In [3]:
#count file records
!wc -l "fhvhv_tripdata_2021-01.csv"

11908469 fhvhv_tripdata_2021-01.csv


In [6]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   NULL|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   NULL|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   NULL|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   NULL|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   NULL|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [4]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None),
 Row(hvfhs_

In [10]:
!head -n 1001 "fhvhv_tripdata_2021-01.csv" > head.csv

In [11]:
!wc -l head.csv

1001 head.csv


In [7]:
import pandas as pd

In [42]:
df_pandas = pd.read_csv("head.csv")
df_pandas.head()

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,NaN
1,HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,NaN
2,HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,NaN
3,HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,NaN
4,HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,NaN


In [43]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

Spark hasn't infer schema

In [8]:
from pyspark.sql import types
#Create User defined Custom Schema using StructType, True means Nullable
mySchema = types.StructType([ types.StructField("hvfhs_license_num", types.StringType(), True)\
                       ,types.StructField("dispatching_base_num", types.StringType(), True)\
                       ,types.StructField("pickup_datetime", types.TimestampType(), True)\
                       ,types.StructField("dropoff_datetime", types.TimestampType(), True)\
                       ,types.StructField("PULocationID", types.IntegerType(), True)\
                       ,types.StructField("DOLocationID", types.IntegerType(), True)\
                       ,types.StructField("SR_Flag", types.FloatType(), True)
                      ])

In [9]:
#read file as spark dataframe
df = spark.read \
    .option("header", "true") \
    .schema(mySchema)\
    .csv('fhvhv_tripdata_2021-01.csv')

df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   NULL|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   NULL|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   NULL|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   NULL|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   NULL|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [10]:
df.dtypes

[('hvfhs_license_num', 'string'),
 ('dispatching_base_num', 'string'),
 ('pickup_datetime', 'timestamp'),
 ('dropoff_datetime', 'timestamp'),
 ('PULocationID', 'int'),
 ('DOLocationID', 'int'),
 ('SR_Flag', 'float')]

- select & filter are transformation(lazy) because of not excuted untill apply action like: show()

In [11]:
df.select('dispatching_base_num', 'pickup_datetime', 'dropoff_datetime')\
.filter(df.hvfhs_license_num == 'HV0003').show()

+--------------------+-------------------+-------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|
+--------------------+-------------------+-------------------+
|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|
|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|
|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|
|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|
|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|
|              B02764|2021-01-01 00:14:30|2021-01-01 00:50:27|
|              B02875|2021-01-01 00:22:54|2021-01-01 00:30:20|
|              B02875|2021-01-01 00:40:12|2021-01-01 00:53:31|
|              B02875|2021-01-01 00:56:45|2021-01-01 01:17:42|
|              B02835|2021-01-01 00:29:04|2021-01-01 00:36:27|
|              B02835|2021-01-01 00:48:56|2021-01-01 00:59:12|
|              B02682|2021-01-01 00:11:53|2021-01-01 00:18:06|
|              B02682|2021-01-01 00:28:31|2021-01-01 00

In [12]:
#import built in functions from pyspark
from pyspark.sql import functions as f
from pyspark.sql.functions import col

In [11]:
#df.withColumn(): it is spark dataframe function used to add a new or update an existing column on DataFrame
#f.to_date pyspark sql function which convert column to date
df\
.withColumn('pickup_date', f.to_date(df.pickup_datetime))\
            .withColumn('dropoff_date', f.to_date(df.dropoff_datetime))\
            .select('pickup_datetime', 'pickup_date', 'dropoff_datetime', 'dropoff_date')\
            .show()

+-------------------+-----------+-------------------+------------+
|    pickup_datetime|pickup_date|   dropoff_datetime|dropoff_date|
+-------------------+-----------+-------------------+------------+
|2021-01-01 00:33:44| 2021-01-01|2021-01-01 00:49:07|  2021-01-01|
|2021-01-01 00:55:19| 2021-01-01|2021-01-01 01:18:21|  2021-01-01|
|2021-01-01 00:23:56| 2021-01-01|2021-01-01 00:38:05|  2021-01-01|
|2021-01-01 00:42:51| 2021-01-01|2021-01-01 00:45:50|  2021-01-01|
|2021-01-01 00:48:14| 2021-01-01|2021-01-01 01:08:42|  2021-01-01|
|2021-01-01 00:06:59| 2021-01-01|2021-01-01 00:43:01|  2021-01-01|
|2021-01-01 00:50:00| 2021-01-01|2021-01-01 01:04:57|  2021-01-01|
|2021-01-01 00:14:30| 2021-01-01|2021-01-01 00:50:27|  2021-01-01|
|2021-01-01 00:22:54| 2021-01-01|2021-01-01 00:30:20|  2021-01-01|
|2021-01-01 00:40:12| 2021-01-01|2021-01-01 00:53:31|  2021-01-01|
|2021-01-01 00:56:45| 2021-01-01|2021-01-01 01:17:42|  2021-01-01|
|2021-01-01 00:29:04| 2021-01-01|2021-01-01 00:36:27|  2021-01

In [13]:
def lower_char(base_number):
   return base_number[0].lower() + base_number[1:]

In [74]:
lower_char('B0276')

'b0276'

In [14]:
from pyspark.sql.functions import udf
#to register upper function as user defined function to use in dataframe or SQL
lower_func = udf(lambda x : lower_char(x), returnType = types.StringType())

In [15]:
df\
.withColumn('dispatching_base', lower_func(col("dispatching_base_num")))\
.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|dispatching_base|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   NULL|          b02682|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   NULL|          b02682|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   NULL|          b02764|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   NULL|          b02764|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08

In [19]:
#partition dataframe into 24 partions
# it's a lazy command, which is run when call it
df = df.repartition(24)

when run below command give me error: Py4JJavaError
Python 3.11 is not yet stable for PySpark. So, make sure you're setting up your virtualenv with either Python 3.9 or Python 3.10
Solution: 
$ dd$ pip install --no-cashe-dir pyspark

In [18]:
df.write.parquet('fhvhv/2021/01_temp/', mode="overwrite")